In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("logistic").master("local[*]").getOrCreate()

In [4]:
data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

In [3]:
from pyspark.ml.classification import LogisticRegression

In [5]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
lr = LogisticRegression(featuresCol='features', labelCol='label')

In [7]:
fited = lr.fit(data)

In [8]:
results = fited.summary

In [11]:
results.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [12]:
results.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111048...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126969...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170264...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200596...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695572...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102604...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215612...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [13]:
train_Data, test_data = data.randomSplit([0.7,0.3])

In [14]:
model = LogisticRegression()

In [15]:
fit_final = model.fit(train_Data)

In [17]:
prediction_and_labels =  fit_final.evaluate(test_data)

In [18]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[27.0595255360660...|[0.99999999999822...|       0.0|
|  0.0|(692,[122,123,124...|[18.1020454659116...|[0.99999998624750...|       0.0|
|  0.0|(692,[122,123,148...|[19.462819281932,...|[0.99999999647300...|       0.0|
|  0.0|(692,[124,125,126...|[43.1117130327934...|[1.0,1.8915634968...|       0.0|
|  0.0|(692,[124,125,126...|[27.6168341584143...|[0.99999999999898...|       0.0|
|  0.0|(692,[126,127,128...|[17.0477197057880...|[0.99999996052979...|       0.0|
|  0.0|(692,[126,127,128...|[26.6407470084160...|[0.99999999999730...|       0.0|
|  0.0|(692,[154,155,156...|[10.3188344994198...|[0.99996699552845...|       0.0|
|  0.0|(692,[234,235,237...|[5.18573740070312...|[0.99443533008828...|       0.0|
|  1.0|(692,[124

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [20]:
my_eval = BinaryClassificationEvaluator()

In [21]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [22]:
my_final_roc

1.0

In [23]:
spark.stop()